In [ ]:
import json
posts = json.load(open('posts.json'))
len(posts), posts[0]

In [ ]:
text = '\n'.join(posts).lower()
len(text)

In [ ]:
characters = sorted(list(set(text)))
char_lookup = characters.index
len(characters)

In [30]:
from tqdm import tqdm

maxlen = 100
X, Y = [], []
for i in tqdm(range(len(text) - maxlen)):
    x = text[i:i+maxlen]
    y = text[i+maxlen]
    X.append([char_lookup(c) for c in x])
    Y.append(char_lookup(y))
len(X), len(Y)

100%|██████████| 147405/147405 [00:20<00:00, 7224.78it/s]


(147405, 147405)

In [31]:
import numpy as np
import keras.utils

X = np.reshape(X, (len(X), maxlen)) / len(characters) # normalized seq input
Y = keras.utils.to_categorical(Y)

X.shape, Y.shape

((147405, 100), (147405, 776))

In [25]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Flatten, Dropout

In [32]:
model = Sequential()
model.add(Embedding(len(characters), 40, input_length=maxlen, name="Embedding"))
model.add(LSTM(256, name="LSTM"))
model.add(Dropout(0.2, name="Dropout"))
model.add(Dense(len(characters), activation='softmax', name="FullyConnected"))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding (Embedding)        (None, 100, 40)           31040     
_________________________________________________________________
LSTM (LSTM)                  (None, 256)               304128    
_________________________________________________________________
Dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
FullyConnected (Dense)       (None, 776)               199432    
Total params: 534,600
Trainable params: 534,600
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X, Y, epochs=1, batch_size=256, shuffle=True)

Epoch 1/1
142592/147405 [============================>.] - ETA: 21s - loss: 3.4439 - acc: 0.1677